In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender']) 

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [6]:
geo_encoded_df = pd.DataFrame(
    geo_encoder,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

In [7]:
data = pd.concat(
    [data.drop('Geography', axis=1), geo_encoded_df],
    axis=1
)

In [8]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [9]:
X = data.drop('Exited' , axis=1)
y = data['Exited']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [13]:
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

import datetime

In [14]:
#build ann model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
]
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Optimizer & Loss Function
# Adam optimizer -> adaptive learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# BinaryCrossentropy -> suitable for binary classification
loss = tf.keras.losses.BinaryCrossentropy()

In [17]:
# Compile the Model
model.compile(
    optimizer=opt,
    loss="binary_crossentropy",
    metrics=['accuracy']
)
# Compile model with optimizer, loss function, and evaluation metric

In [18]:
# TensorBoard Setup
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Create unique log directory for TensorBoard

In [19]:
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)
# Enable visualization of weights & metrics

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

# Early Stopping
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
# Stop training if validation loss doesn't improve

In [21]:
# Train the Model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback]
)
# Train ANN model with validation monitoring


Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 0.3966 - accuracy: 0.8341 - val_loss: 0.3672 - val_accuracy: 0.8545
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3562 - accuracy: 0.8525 - val_loss: 0.3418 - val_accuracy: 0.8625
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3477 - accuracy: 0.8593 - val_loss: 0.3424 - val_accuracy: 0.8600
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3449 - accuracy: 0.8611 - val_loss: 0.3353 - val_accuracy: 0.8620
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3404 - accuracy: 0.8572 - val_loss: 0.3370 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3393 - accuracy: 0.8610 - val_loss: 0.3383 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3363 - accuracy: 0.8635 - val_loss: 0.3337 - val_accuracy: 0.85

In [22]:
# Save the Trained Model
model.save('model.h5')
# Save trained ANN model for future use

c:\Users\Satyabrata\OneDrive\Desktop\ANN\annenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
# TensorBoard Visualization
%load_ext tensorboard
# Load TensorBoard extension in Jupyter

In [24]:
%tensorboard --logdir logs/fit
# Launch TensorBoard to visualize training process